<a href="https://colab.research.google.com/github/BhargaviBathini/AI-DUBBING/blob/main/AI_DUBBING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install openai-whisper ffmpeg gtts transformers moviepy gradio huggingface_hub


In [15]:
import os
import whisper
import torch
import gradio as gr
from gtts import gTTS
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from moviepy.editor import VideoFileClip, AudioFileClip
from huggingface_hub import login


In [16]:
login()


In [17]:
model_whisper = whisper.load_model("base")


  checkpoint = torch.load(fp, map_location=device)



In [18]:
def transcribe_audio(audio_path):
    result = model_whisper.transcribe(audio_path)
    return result["text"]


In [19]:
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_translate = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [20]:
def translate_text(text, target_lang):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    inputs["forced_bos_token_id"] = tokenizer.convert_tokens_to_ids(target_lang)
    translated = model_translate.generate(**inputs)
    return tokenizer.batch_decode(translated, skip_special_tokens=True)[0]


In [21]:
def generate_speech(text, language_code, output_audio_path):
    tts = gTTS(text, lang=language_code)
    tts.save(output_audio_path)


In [22]:
def extract_audio_from_video(video_path, audio_output_path="extracted_audio.mp3"):
    os.system(f"ffmpeg -i {video_path} -q:a 0 -map a {audio_output_path} -y")
    return audio_output_path


In [23]:
def merge_audio_with_video(video_path, audio_path, output_video_path="dubbed_video.mp4"):
    os.system(f"ffmpeg -i {video_path} -i {audio_path} -c:v copy -map 0:v:0 -map 1:a:0 {output_video_path} -y")
    return output_video_path


In [24]:
def process_file(file_path, is_video, target_lang_code, tts_lang_code):
    if is_video:
        audio_path = extract_audio_from_video(file_path)
    else:
        audio_path = file_path

    # Transcribe
    transcribed_text = transcribe_audio(audio_path)

    # Translate
    translated_text = translate_text(transcribed_text, target_lang_code)

    # Generate Speech
    output_audio_path = "dubbed_audio.mp3"
    generate_speech(translated_text, tts_lang_code, output_audio_path)

    # Merge audio with video if input is a video
    if is_video:
        output_video_path = merge_audio_with_video(file_path, output_audio_path)
        return output_video_path  # Return dubbed video
    else:
        return output_audio_path  # Return dubbed audio


In [25]:
lang_mapping = {
    "Telugu": ("tel_Telu", "te"),
    "Hindi": ("hin_Deva", "hi"),
    "Spanish": ("spa_Latn", "es"),
    "French": ("fra_Latn", "fr"),
    "German": ("deu_Latn", "de"),
    "Chinese": ("zho_Hans", "zh"),
}


In [26]:
def main(file, language):
    if language not in lang_mapping:
        return "Language not supported."

    target_lang_code, tts_lang_code = lang_mapping[language]

    # Detect if file is video or audio
    is_video = file.name.endswith((".mp4", ".avi", ".mov", ".mkv"))

    # Process file
    output_file = process_file(file.name, is_video, target_lang_code, tts_lang_code)

    return output_file

iface = gr.Interface(
    fn=main,
    inputs=[
        gr.File(label="Upload Audio/Video File"),
        gr.Dropdown(["Telugu", "Hindi", "Spanish", "French", "German", "Chinese"], label="Select Language"),
    ],
    outputs=gr.File(label="Download Dubbed File"),
    title="AI-Powered Video & Audio Dubbing",
    description="Upload an audio or video file, select a target language, and get a dubbed version!"
)

iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b248f0ae942c44210c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
